In [66]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
from pyts.classification import BOSSVS
from pyts.multivariate.classification import MultivariateClassifier
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame
import sktime
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
import cesium

In [78]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [79]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)     

In [80]:
# Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
        if lg==binning:
            idx_ON.append(i)


    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [81]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []

nbfeatures=5

a=binning
b=nbfeatures
c=lg

# multivariate

data_matrix= np.zeros((b,a))
on=[]
off=[]
photon_idx=[]
delta_index=[]
flux=[]
delta_flux=[]
mjd=[]
fratio=[]
flux_idx=[]
flux_error=[]
#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    photon_idx.append(np.array(dataframe['Photon Index']))
    delta_index.append(np.array(dataframe['Delta Index']))
    mjd.append(np.array(dataframe['MJD']))
    flux.append(np.array(dataframe['Flux']))
    delta_flux.append(np.array(dataframe['Delta Flux']))
    flux_idx.append(np.array(dataframe['Flux'])*np.array(dataframe['Photon Index']))
    fratio.append(np.array(dataframe['fratio']))
    flux_error.append(np.array(dataframe['Delta Flux']))
for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    photon_idx.append(np.array(dataframe['Photon Index']))
    delta_index.append(np.array(dataframe['Delta Index']))
    mjd.append(np.array(dataframe['MJD']))
    flux.append(np.array(dataframe['Flux']))
    flux_idx.append(np.array(dataframe['Flux'])*np.array(dataframe['Photon Index']))
    fratio.append(np.array(dataframe['fratio']))
    flux_error.append(np.array(dataframe['Delta Flux']))
#Creating labels
for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    Labels.append(int(1))


In [10]:
import numpy as np
import cesium
from cesium import featurize
import warnings
warnings.filterwarnings("ignore")


features_to_use = [
                  'std','amplitude','max_slope']
# features_to_use = [
#                   "max_slope",
#                   "med_err","stetson_j","std_err"]
fset_cesium = featurize.featurize_time_series(times=mjd,
                                              values=photon_idx,
                                              errors=delta_index,
                                              features_to_use=features_to_use)
print(fset_cesium)

feature       std amplitude max_slope
channel         0         0         0
0        0.235597  0.544947  0.008073
1        0.612850  1.515464  0.020950
2        0.530785  1.571777  0.019666
3        0.304529  0.809423  0.009043
4        0.760772  1.860631  0.021070
...           ...       ...       ...
1135     1.096254  2.499288  0.030368
1136     0.252557  0.710988  0.008264
1137     0.506804  1.582686  0.022016
1138     1.118603  2.282258  0.021814
1139     0.639641  2.078685  0.023031

[1140 rows x 3 columns]


In [90]:
import numpy as np
import cesium
from cesium import featurize
import warnings
warnings.filterwarnings("ignore")



# features_to_use = [
#                   'std','percent_amplitude','amplitude','max_slope']

features_to_use = [
                  'percent_difference_flux_percentile']
fset_cesium = featurize.featurize_time_series(times=mjd,
                                              values=flux, 
                                              errors=flux_error,
                                              features_to_use=features_to_use)
print(fset_cesium )

feature percent_difference_flux_percentile
channel                                  0
0                             2.331179e-08
1                             2.632709e-08
2                             1.639528e-08
3                             1.006381e-08
4                             2.119383e-08
...                                    ...
1135                          1.948568e-08
1136                          1.483723e-08
1137                          5.946276e-08
1138                          2.156158e-07
1139                          1.911282e-08

[1140 rows x 1 columns]


In [76]:
features_to_use = [
                  'std','percent_amplitude']
fset_cesium = featurize.featurize_time_series(times=mjd,
                                              values=photon_idx,
                                              errors=delta_index,
                                              features_to_use=features_to_use)
print(fset_cesium)

feature       std percent_amplitude
channel         0                 0
0        0.813682          6.115137
1        0.324348          0.738986
2        0.320741          1.442672
3        0.427394          3.434630
4        0.659141          4.826915
..            ...               ...
119      0.283727          1.399212
120      0.639092          7.111547
121      0.972994          5.631823
122      0.527917          2.911923
123      0.310025          1.432305

[124 rows x 2 columns]


In [91]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
bad_ON=[]
iterations=5
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
for i in  range(iterations):
                 
    train, test = train_test_split(np.arange(lg), random_state=i)

    model_cesium = RandomForestClassifier(n_estimators=100, max_features="auto",
                                          random_state=0,class_weight =class_weight)
    Labels=np.hstack(Labels)
    model_cesium.fit(fset_cesium.iloc[train], Labels[train])
    prediction= model_cesium.predict(fset_cesium.iloc[test])
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    y_test=Labels[test]
    for i in range(len(y_test)):
        if y_test[i]==1 :
            on_nbs+=1
            if prediction[i]==1:
                on_score+=1
                
            else : 
                fon+=1
                bad_ON.append(i)
        if y_test[i]==0 :
            off_nbs+=1
            if prediction[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(prediction,y_test,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Accuracy for ON class:  80.0 %
Accuracy for OFF class:  20.651650708941556 %
False Positive rate:  20.0 %
False Negative rate:  79.34834929105844 %
F1 score:  0.24236496509802313


In [92]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

train, test = train_test_split(np.arange(lg), random_state=0)

model_cesium = KNeighborsClassifier(2)
Labels=np.hstack(Labels)
model_cesium.fit(fset_cesium.iloc[train], Labels[train])

KNeighborsClassifier(n_neighbors=2)

In [93]:
from sklearn.metrics import accuracy_score

train, test = train_test_split(np.arange(lg), random_state=0)
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
bad_ON=[]
iterations=5
lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
prediction = model_cesium.predict(fset_cesium.iloc[test])
on_score=0
on_nbs=0
off_nbs=0
off_score=0
foff=0
fon=0
y_test=Labels[test]
for i in range(len(y_test)):
    if y_test[i]==1 :
        on_nbs+=1
        if prediction[i]==1:
            on_score+=1

        else : 
            fon+=1
            bad_ON.append(i)
    if y_test[i]==0 :
        off_nbs+=1
        if prediction[i]==0:
            off_score+=1 
        else:
            foff+=1
if on_nbs>0:    
    ON_accuracy.append(100*(on_score/on_nbs))
    FPOS.append(100*(fon/on_nbs))
OFF_accuracy.append(100*(off_score/off_nbs))
FNEG.append(100*(foff/off_nbs))
f1= f1_score(prediction,y_test,average='weighted')
fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Accuracy for ON class:  0.0 %
Accuracy for OFF class:  100.0 %
False Positive rate:  100.0 %
False Negative rate:  0.0 %
F1 score:  0.983957219251337
